In [3]:
!pip install darts

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 592.0/592.0 KB 17.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.4/9.4 MB 99.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.0/92.0 KB 12.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 81.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 89.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 112.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.7/147.7 KB 19.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.6/76.6 MB 12.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 575.9/575.9 KB 53.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 825.8/825.8 KB 53.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import numpy as np
import pandas as pd
import darts
from darts import TimeSeries
import warnings
warnings.filterwarnings('ignore')
import matplotlib.pyplot as plt
#import seaborn as sns
from datetime import datetime
from darts.dataprocessing.transformers import Scaler
from darts.metrics import smape, r2_score, rmse

from darts.models import (
    AutoARIMA,
    Prophet,
    RNNModel,
    NBEATSModel,
    BlockRNNModel,
    TFTModel,
    TransformerModel
)

import torch
from tqdm import tqdm
import numpy as np
from darts.utils.likelihood_models import QuantileRegression

In [3]:
df = pd.read_csv('/content/drive/MyDrive/train.csv', parse_dates=['Date'])
df.sample(10)
df = pd.concat([df.drop(columns='StateHoliday'), pd.get_dummies(df.StateHoliday, prefix='Holiday')], axis=1)

In [4]:
def get_sample(num):
    sample = df[df.Store == num]
    series = TimeSeries.from_dataframe(sample, 'Date', 'Sales')

    train, test = series.split_before(pd.Timestamp("20150601"))

    transformer = Scaler()
    train_transformed = transformer.fit_transform(train)
    test_transformed = transformer.transform(test)

    series_customers = TimeSeries.from_dataframe(sample, time_col='Date', value_cols='Customers')
    series_open = TimeSeries.from_dataframe(sample, time_col='Date', value_cols='Open')
    series_promo = TimeSeries.from_dataframe(sample, time_col='Date', value_cols='Promo')
    series_school = TimeSeries.from_dataframe(sample, time_col='Date', value_cols='SchoolHoliday')
    series_weekday = TimeSeries.from_dataframe(sample, time_col='Date', value_cols='DayOfWeek')
    series_holiday_a = TimeSeries.from_dataframe(sample, time_col='Date', value_cols='Holiday_a')
    series_holiday_b = TimeSeries.from_dataframe(sample, time_col='Date', value_cols='Holiday_b')
    series_holiday_c = TimeSeries.from_dataframe(sample, time_col='Date', value_cols='Holiday_c')

    customers_transformed = transformer.transform(series_customers)

    covariates = series_customers.stack(series_open)
    covariates = covariates.stack(series_promo)
    covariates = covariates.stack(series_school)
    covariates = covariates.stack(series_weekday)
    covariates = covariates.stack(series_holiday_a)
    covariates = covariates.stack(series_holiday_b)
    covariates = covariates.stack(series_holiday_c)

    train_covariates, test_covariates = covariates.split_before(pd.Timestamp("20150601"))

    return train_transformed, test_transformed, covariates

In [5]:
train, test, covariates = get_sample(10)

In [6]:
def rmspe(actual, pred):
    return np.sqrt(np.mean( ((actual - pred) / actual)**2)) 

def smape(actual, pred):
    return np.mean(np.abs(pred - actual) / ((np.abs(actual) + np.abs(pred))/2)) * 100

def evaluate_model(model, train, test):
    model.fit(train)
    pred = model.predict(len(test))
    test = test.pd_dataframe()
    pred = pred.pd_dataframe()

    test.columns = ['Actual']
    pred.columns = ['Pred']

    df = pd.concat([test, pred], axis=1)
    df = df[df.Actual > 0]
    return smape(df.Actual.values, df.Pred.values), rmspe(df.Actual.values, df.Pred.values)

In [7]:
store_num_list = pd.read_csv('/content/drive/MyDrive/store_list.csv').Store.values

## 1. Transfer Model

In [8]:
def evaluate_covariates_model(model, train, test, covariates=None):
    if covariates is not None:
        model.fit(
            train,
            future_covariates=covariates,
            verbose=False,
        )
    else:
        model.fit(
            train,
            verbose=False,
        )
    if covariates is not None:
        pred = model.predict(len(test), future_covariates=covariates)
    else:
        pred = model.predict(len(test))
    pred = pred.pd_dataframe()
    test = test.pd_dataframe()

    test.columns = ['Actual']
    pred.columns = ['Pred']

    df = pd.concat([test, pred], axis=1)
    df = df[df.Actual > 0]
    return smape(df.Actual.values, df.Pred.values), rmspe(df.Actual.values, df.Pred.values)

In [9]:
quantiles = [
    0.01,
    0.05,
    0.1,
    0.15,
    0.2,
    0.25,
    0.3,
    0.4,
    0.5,
    0.6,
    0.7,
    0.75,
    0.8,
    0.85,
    0.9,
    0.95,
    0.99,
]

In [ ]:
smape_list = []
rmspe_list = []

for idx, num in enumerate(store_num_list):
    if idx % 2 == 1:
        continue
    train_transformed, test_transformed, covariates = get_sample(num)
    tft_model = TFTModel(
        input_chunk_length=28,
        output_chunk_length=7,
        hidden_size=64,
        lstm_layers=1,
        num_attention_heads=4,
        dropout=0.1,
        batch_size=16,
        n_epochs=300,
        add_relative_index=False,
        add_encoders=None,
        likelihood=QuantileRegression(
                quantiles=quantiles
        ),  
        random_state=42, 
        pl_trainer_kwargs ={"accelerator": "gpu", "devices": [0], "enable_progress_bar": False}
    )
    
    smape_score, rmspe_score = evaluate_covariates_model(tft_model, train_transformed, test_transformed, covariates )
    rmspe_list.append(rmspe_score)
    smape_list.append(smape_score)
    print(num, smape_score, rmspe_score)

INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightni

10 8.094236439846897 0.1037844591387345


In [ ]:
smape1_list = []
rmspe1_list = []

for num in tqdm(store_num_list):
    train_transformed, test_transformed, covariates = get_sample(num)
    tft_model = TFTModel(
        input_chunk_length=28,
        output_chunk_length=7,
        hidden_size=64,
        lstm_layers=1,
        num_attention_heads=4,
        dropout=0.1,
        batch_size=16,
        n_epochs=300,
        add_relative_index=False,
        add_encoders=None,
        likelihood=QuantileRegression(
                quantiles=quantiles
        ),  
        random_state=42, 
        pl_trainer_kwargs ={"accelerator": "gpu", "devices": [0], "enable_progress_bar": False}
    )
    
    smape_score, rmspe_score = evaluate_covariates_model(tft_model, train_transformed, test_transformed, covariates )
    rmspe1_list.append(rmspe_score)
    smape1_list.append(smape_score)
    print(num, smape_score, rmspe_score)

In [ ]:
print(np.mean(smape_list), np.std(smape_list), np.max(smape_list), np.min(smape_list))
plt.hist(smape_list)

In [ ]:
print(np.mean(rmspe_list), np.std(rmspe_list), np.max(rmspe_list), np.min(rmspe_list))
plt.hist(rmspe_list)

In [ ]:
res = pd.DataFrame(
    {
        'without_rmspe' : rmspe_list,
        'without_smape' : smape_list,
        #'with_rmspe' : rmspe_list,
        #'with_smape' : smape_list,
    }
)

### 1.3 Result Comparison

In [ ]:
result = res.copy()
result['Store'] = store_num_list
result.to_csv('/content/drive/MyDrive/transformer_nocov_result.csv')
result.head(10)

In [ ]:
train_transformed, test_transformed, covariates = get_sample(1)
tft_model = TFTModel(
    input_chunk_length=28,
    output_chunk_length=7,
    hidden_size=64,
    lstm_layers=1,
    num_attention_heads=4,
    dropout=0.1,
    batch_size=16,
    n_epochs=300,
    add_relative_index=False,
    add_encoders=None,
    likelihood=QuantileRegression(
        quantiles=quantiles
    ),  
    random_state=42, 
    pl_trainer_kwargs ={"accelerator": "gpu", "devices": [0], "enable_progress_bar": False}
)

tft_model.fit(train_transformed)
pred_demand_covs = tft_model.predict(len(test_transformed))

plt.figure(figsize=(15, 6))
test_transformed.plot(label="actual")
pred_demand_covs.plot(label="forecast")
plt.title('Generic N-Beats')
plt.show()

In [ ]:
train_transformed, test_transformed, covariates = get_sample(10)
tft_model = TFTModel(
    input_chunk_length=28,
    output_chunk_length=7,
    hidden_size=64,
    lstm_layers=1,
    num_attention_heads=4,
    dropout=0.1,
    batch_size=16,
    n_epochs=300,
    add_relative_index=False,
    add_encoders=None,
    likelihood=QuantileRegression(
        quantiles=quantiles
    ),  
    random_state=42, 
    pl_trainer_kwargs ={"accelerator": "gpu", "devices": [0], "enable_progress_bar": False}
)

tft_model.fit(train_transformed, future_covariates=covariates)
pred_demand_covs = tft_model.predict(len(test_transformed), future_covariates=covariates)

plt.figure(figsize=(15, 6))
test_transformed.plot(label="actual")
pred_demand_covs.plot(label="forecast")
plt.title('TFT')
plt.show()

## Hyper Parameter Tune

In [11]:
import pandas as pd
from pytorch_lightning.callbacks import EarlyStopping
from ray import tune
from ray.tune import CLIReporter
from ray.tune.integration.pytorch_lightning import TuneReportCallback
from ray.tune.schedulers import ASHAScheduler
from torchmetrics import MeanAbsoluteError, MeanAbsolutePercentageError, MetricCollection, SymmetricMeanAbsolutePercentageError

In [12]:
from darts.utils.likelihood_models import GaussianLikelihood

def train_model(model_args, callbacks, train, val, covariates):
    torch_metrics = MetricCollection([SymmetricMeanAbsolutePercentageError(), MeanAbsoluteError()])
    # Create the model using model_args from Ray Tune
    model = TFTModel(
        input_chunk_length=7,
        output_chunk_length=7,
        lstm_layers=1,
        num_attention_heads=4,
        n_epochs=300,
        add_relative_index=False,
        add_encoders=None,
        likelihood=QuantileRegression(
                quantiles=quantiles
        ),  
        random_state=42, 
        torch_metrics=torch_metrics,
        pl_trainer_kwargs ={"callbacks": callbacks, "enable_progress_bar": False},
        **model_args
    )

    model.fit(
        series=train,
        val_series=val,
        future_covariates=covariates,
        val_future_covariates=covariates
    )

In [13]:
train, test, covariates = get_sample(10)
train, val = train.split_after(pd.Timestamp(year=2015, month=4, day=1))

In [14]:
my_stopper = EarlyStopping(
    monitor="val_SymmetricMeanAbsolutePercentageError",
    patience=5,
    min_delta=0.05,
    mode='min',
)

# set up ray tune callback
tune_callback = TuneReportCallback(
    {
        "loss": "val_Loss",
        "SMAPE": "val_SymmetricMeanAbsolutePercentageError",
    },
    on="validation_end",
)

# define the hyperparameter space
config = {
    #"input_chunk_length": tune.choice([7, 28, 30]),
    #"output_chunk_length": tune.choice([1, 7]),
    "batch_size": tune.choice([16, 32, 64, 128]),
    "hidden_size": tune.choice([32, 48, 64]),
    "dropout": tune.uniform(0.1, 0.3),
}

reporter = CLIReporter(
    parameter_columns=list(config.keys()),
    metric_columns=["loss", "SMAPE", "training_iteration"],
)

resources_per_trial = {"cpu": 12, "gpu": 1}

# the number of combinations to try
num_samples = 20

scheduler = ASHAScheduler(max_t=1000, grace_period=3, reduction_factor=2)

train_fn_with_parameters = tune.with_parameters(
    train_model, callbacks=[my_stopper, tune_callback], train=train, val=val, covariates=covariates
)

# optimize hyperparameters by minimizing the SMAPE on the validation set
analysis = tune.run(
    train_fn_with_parameters,
    resources_per_trial=resources_per_trial,
    # Using a metric instead of loss allows for
    # comparison between different likelihood or loss functions.
    metric="SMAPE",  # any value in TuneReportCallback.
    mode="min",
    config=config,
    num_samples=num_samples,
    scheduler=scheduler,
    progress_reporter=reporter,
    name="tune_darts",
)

print("Best hyperparameters found were: ", analysis.best_config)

2023-01-30 19:27:44,947	INFO worker.py:1538 -- Started a local Ray instance.


== Status ==
Current time: 2023-01-30 19:27:46 (running for 00:00:00.24)
Memory usage on this node: 3.4/83.5 GiB 
Using AsyncHyperBand: num_stopped=0
Bracket: Iter 768.000: None | Iter 384.000: None | Iter 192.000: None | Iter 96.000: None | Iter 48.000: None | Iter 24.000: None | Iter 12.000: None | Iter 6.000: None | Iter 3.000: None
Resources requested: 12.0/12 CPUs, 1.0/1 GPUs, 0.0/49.84 GiB heap, 0.0/24.92 GiB objects
Result logdir: /root/ray_results/tune_darts
Number of trials: 16/20 (15 PENDING, 1 RUNNING)
+-------------------------+----------+-------------------+--------------+---------------+-----------+
| Trial name              | status   | loc               |   batch_size |   hidden_size |   dropout |
|-------------------------+----------+-------------------+--------------+---------------+-----------|
| train_model_2cb17_00000 | RUNNING  | 172.28.0.12:55343 |           32 |            64 |  0.127941 |
| train_model_2cb17_00001 | PENDING  |                   |           32 |

(train_model pid=55343) GPU available: True (cuda), used: False
(train_model pid=55343) TPU available: False, using: 0 TPU cores
(train_model pid=55343) IPU available: False, using: 0 IPUs
(train_model pid=55343) HPU available: False, using: 0 HPUs
(train_model pid=55343) /usr/local/lib/python3.8/dist-packages/pytorch_lightning/trainer/setup.py:175: PossibleUserWarning: GPU available but not used. Set `accelerator` and `devices` using `Trainer(accelerator='gpu', devices=1)`.
(train_model pid=55343)   rank_zero_warn(
(train_model pid=55343) 
(train_model pid=55343)    | Name                              | Type                             | Params
(train_model pid=55343) ----------------------------------------------------------------------------------------
(train_model pid=55343) 0  | train_metrics                     | MetricCollection                 | 0     
(train_model pid=55343) 1  | val_metrics                       | MetricCollection                 | 0     
(train_model pid=55

Trial name,SMAPE,date,done,episodes_total,experiment_id,hostname,iterations_since_restore,node_ip,pid,time_since_restore,time_this_iter_s,time_total_s,timestamp,timesteps_since_restore,timesteps_total,training_iteration,trial_id,warmup_time
train_model_2cb17_00000,0.522666,2023-01-30_19-28-16,True,,7360d7610368490992a6e91342cc48a3,ee5d02e41edb,10,172.28.0.12,55343,24.6291,2.19586,24.6291,1675106896,0,,10,2cb17_00000,0.00367045
train_model_2cb17_00001,0.551715,2023-01-30_19-28-44,True,,d2ca005f2b7b4918ab637e3da9dd51c5,ee5d02e41edb,10,172.28.0.12,55576,22.1769,1.95423,22.1769,1675106924,0,,10,2cb17_00001,0.00333333
train_model_2cb17_00002,0.532596,2023-01-30_19-29-16,True,,69d18292901546caa6dc6592dac8858c,ee5d02e41edb,12,172.28.0.12,55792,26.0517,1.93021,26.0517,1675106956,0,,12,2cb17_00002,0.00321722
train_model_2cb17_00003,0.689665,2023-01-30_19-29-28,True,,f23dd23ea901410993f23c902111fa73,ee5d02e41edb,3,172.28.0.12,56028,6.28707,1.26671,6.28707,1675106968,0,,3,2cb17_00003,0.00336671
train_model_2cb17_00004,0.783591,2023-01-30_19-29-39,True,,9d99d25a763b4f9a8d499af34d6a9ed2,ee5d02e41edb,3,172.28.0.12,56165,5.52578,0.999355,5.52578,1675106979,0,,3,2cb17_00004,0.00322628
train_model_2cb17_00005,0.539846,2023-01-30_19-30-09,True,,80134474dd78486eb0ffee0cb62ac5e1,ee5d02e41edb,11,172.28.0.12,56295,23.9467,1.94528,23.9467,1675107009,0,,11,2cb17_00005,0.00309873
train_model_2cb17_00006,0.688991,2023-01-30_19-30-22,True,,eb750046d6ee40fa99429dca1bdc4b07,ee5d02e41edb,3,172.28.0.12,56520,6.89544,1.45701,6.89544,1675107022,0,,3,2cb17_00006,0.00323153
train_model_2cb17_00007,0.789988,2023-01-30_19-30-34,True,,62bb72fb4d844d078c861290da773081,ee5d02e41edb,3,172.28.0.12,56663,6.6142,1.35689,6.6142,1675107034,0,,3,2cb17_00007,0.00349879
train_model_2cb17_00008,0.556069,2023-01-30_19-30-53,True,,0b7147f195024bee969fe125481e6934,ee5d02e41edb,6,172.28.0.12,56801,13.1521,1.74976,13.1521,1675107053,0,,6,2cb17_00008,0.00314236
train_model_2cb17_00009,0.526454,2023-01-30_19-31-26,True,,93ca7f7c713941748b9a3396fc25fc3d,ee5d02e41edb,9,172.28.0.12,56972,27.4255,2.73378,27.4255,1675107086,0,,9,2cb17_00009,0.00331306


== Status ==
Current time: 2023-01-30 19:27:56 (running for 00:00:10.05)
Memory usage on this node: 5.1/83.5 GiB 
Using AsyncHyperBand: num_stopped=0
Bracket: Iter 768.000: None | Iter 384.000: None | Iter 192.000: None | Iter 96.000: None | Iter 48.000: None | Iter 24.000: None | Iter 12.000: None | Iter 6.000: None | Iter 3.000: None
Resources requested: 12.0/12 CPUs, 1.0/1 GPUs, 0.0/49.84 GiB heap, 0.0/24.92 GiB objects
Current best trial: 2cb17_00000 with SMAPE=0.7104479326208002 and parameters={'batch_size': 32, 'hidden_size': 64, 'dropout': 0.12794087613403066}
Result logdir: /root/ray_results/tune_darts
Number of trials: 17/20 (16 PENDING, 1 RUNNING)
+-------------------------+----------+-------------------+--------------+---------------+-----------+----------+----------------------+
| Trial name              | status   | loc               |   batch_size |   hidden_size |   dropout |    SMAPE |   training_iteration |
|-------------------------+----------+-------------------+----

...

== Status ==
Current time: 2023-01-30 19:34:23 (running for 00:06:37.04)
Memory usage on this node: 5.5/83.5 GiB 
Using AsyncHyperBand: num_stopped=10
Bracket: Iter 768.000: None | Iter 384.000: None | Iter 192.000: None | Iter 96.000: None | Iter 48.000: None | Iter 24.000: None | Iter 12.000: -0.5325960201874448 | Iter 6.000: -0.5441701035779936 | Iter 3.000: -0.6291757046076478
Resources requested: 12.0/12 CPUs, 1.0/1 GPUs, 0.0/49.84 GiB heap, 0.0/24.92 GiB objects
Current best trial: 2cb17_00014 with SMAPE=0.504148006038673 and parameters={'batch_size': 16, 'hidden_size': 64, 'dropout': 0.1048610286049492}
Result logdir: /root/ray_results/tune_darts
Number of trials: 20/20 (1 PENDING, 1 RUNNING, 18 TERMINATED)
+-------------------------+------------+-------------------+--------------+---------------+-----------+----------+----------------------+
| Trial name              | status     | loc               |   batch_size |   hidden_size |   dropout |    SMAPE |   training_iteration |


(train_model pid=58634) 2023-01-30 19:34:23,474	WARNING pytorch_lightning.py:134 -- Metric val_Loss does not exist in `trainer.callback_metrics.
(train_model pid=58634) 2023-01-30 19:34:24,855	WARNING pytorch_lightning.py:134 -- Metric val_Loss does not exist in `trainer.callback_metrics.


== Status ==
Current time: 2023-01-30 19:34:30 (running for 00:06:43.77)
Memory usage on this node: 4.1/83.5 GiB 
Using AsyncHyperBand: num_stopped=11
Bracket: Iter 768.000: None | Iter 384.000: None | Iter 192.000: None | Iter 96.000: None | Iter 48.000: None | Iter 24.000: None | Iter 12.000: -0.5325960201874448 | Iter 6.000: -0.5441701035779936 | Iter 3.000: -0.6364470690502947
Resources requested: 12.0/12 CPUs, 1.0/1 GPUs, 0.0/49.84 GiB heap, 0.0/24.92 GiB objects
Current best trial: 2cb17_00014 with SMAPE=0.504148006038673 and parameters={'batch_size': 16, 'hidden_size': 64, 'dropout': 0.1048610286049492}
Result logdir: /root/ray_results/tune_darts
Number of trials: 20/20 (1 RUNNING, 19 TERMINATED)
+-------------------------+------------+-------------------+--------------+---------------+-----------+----------+----------------------+
| Trial name              | status     | loc               |   batch_size |   hidden_size |   dropout |    SMAPE |   training_iteration |
|----------

(train_model pid=58771) GPU available: True (cuda), used: False
(train_model pid=58771) TPU available: False, using: 0 TPU cores
(train_model pid=58771) IPU available: False, using: 0 IPUs
(train_model pid=58771) HPU available: False, using: 0 HPUs
(train_model pid=58771) /usr/local/lib/python3.8/dist-packages/pytorch_lightning/trainer/setup.py:175: PossibleUserWarning: GPU available but not used. Set `accelerator` and `devices` using `Trainer(accelerator='gpu', devices=1)`.
(train_model pid=58771)   rank_zero_warn(
(train_model pid=58771) 
(train_model pid=58771)    | Name                              | Type                             | Params
(train_model pid=58771) ----------------------------------------------------------------------------------------
(train_model pid=58771) 0  | train_metrics                     | MetricCollection                 | 0     
(train_model pid=58771) 1  | val_metrics                       | MetricCollection                 | 0     
(train_model pid=58

== Status ==
Current time: 2023-01-30 19:34:35 (running for 00:06:49.28)
Memory usage on this node: 5.4/83.5 GiB 
Using AsyncHyperBand: num_stopped=12
Bracket: Iter 768.000: None | Iter 384.000: None | Iter 192.000: None | Iter 96.000: None | Iter 48.000: None | Iter 24.000: None | Iter 12.000: -0.5325960201874448 | Iter 6.000: -0.5441701035779936 | Iter 3.000: -0.6454252438354487
Resources requested: 0/12 CPUs, 0/1 GPUs, 0.0/49.84 GiB heap, 0.0/24.92 GiB objects
Current best trial: 2cb17_00014 with SMAPE=0.504148006038673 and parameters={'batch_size': 16, 'hidden_size': 64, 'dropout': 0.1048610286049492}
Result logdir: /root/ray_results/tune_darts
Number of trials: 20/20 (20 TERMINATED)
+-------------------------+------------+-------------------+--------------+---------------+-----------+----------+----------------------+
| Trial name              | status     | loc               |   batch_size |   hidden_size |   dropout |    SMAPE |   training_iteration |
|-------------------------+

In [15]:
torch.set_float32_matmul_precision('medium')

In [13]:
store_num_list[64]

861

In [14]:
smape_list = []
rmspe_list = []

for num in tqdm(store_num_list[65:]):
    train_transformed, test_transformed, covariates = get_sample(num)
    tft_model = TFTModel(
        input_chunk_length=28,
        output_chunk_length=7,
        hidden_size=64,
        lstm_layers=1,
        num_attention_heads=4,
        dropout= 0.1,
        batch_size=16,
        n_epochs=300,
        add_relative_index=False,
        add_encoders=None,
        likelihood=QuantileRegression(
                quantiles=quantiles
        ),  
        random_state=42, 
        pl_trainer_kwargs ={"accelerator": "gpu", "devices": [0], "enable_progress_bar": False}
    )
    smape_score, rmspe_score = evaluate_covariates_model(tft_model, train_transformed, test_transformed, covariates )
    rmspe_list.append(rmspe_score)
    smape_list.append(smape_score)
    print(num, smape_score, rmspe_score)

  0%|          | 0/35 [00:00<?, ?it/s]INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:lightning_fabric.accelerators.cuda:You are using a CUDA device ('NVIDIA A100-SXM4-40GB') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=300` reached.
INFO:pytorch_lightning.utilities.rank_zero:GPU available: Tru

968 9.24200405313524 0.12143650325133076


INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:lightning_fabric.accelerators.cuda:You are using a CUDA device ('NVIDIA A100-SXM4-40GB') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=300` reached.
INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_ligh

978 10.190973515163423 0.15296954963507944


INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=300` reached.
INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:lightning_fabric.accelerators.cuda:You are using a CUDA device ('NVIDIA A100-SXM4-40GB') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
  9%|▊         | 3/35 [58:36<10:23:18, 1168.71s/it]INFO:pytorch_lightning.utilities.rank_zero:GPU a

...

1046 6.846393681576143 0.08918072868779373


INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=300` reached.
INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:lightning_fabric.accelerators.cuda:You are using a CUDA device ('NVIDIA A100-SXM4-40GB') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
 97%|█████████▋| 34/35 [11:00:07<19:52, 1192.21s/it]

1089 7.866645663111199 0.09115110245040546


INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:lightning_fabric.accelerators.cuda:You are using a CUDA device ('NVIDIA A100-SXM4-40GB') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=300` reached.
INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_ligh

1101 5.91791156003411 0.07550253919020407


In [15]:
res_h1 = pd.DataFrame(
    {
        'rmspe' : rmspe_list,
        'smape' : smape_list,
    }
)
res_h1['Store'] = store_num_list
res_h1

ValueError: ignored

In [ ]:
result = res_h1.set_index('Store')
result = pd.merge(result, store[['Store','StoreType']], left_on='Store', right_on='Store')
result.to_csv('/content/drive/MyDrive/tft_hyper_result.csv')
result.mean()